In [1]:
__author__ = "Philipp Kats"
__date__ = "2016_02_01"

import datetime
import dateutil.parser
import time
import os
import gzip
import json

import sqlite3

In [2]:
PWD = os.getenv('PWD')

path = '/'.join(PWD.split('/')[:-1]) + '/gnip/enviroment4gnip/job1/PTDataDownload/downloads/'

In [3]:
def getFiles(folder, format, full=True):
    for file in os.listdir(folder):
        if file.endswith(format):
            if full:
                yield os.path.join(folder, file)
            else:
                yield file

In [4]:
files = getFiles(path, '.json.gz', full=True)

In [5]:


def parseJsonArrayFile(path):
    with gzip.open(path, 'rb') as f:
        lines = f.read().split('\r')
        records = []
        for line in lines:
            if len(line.strip())>0:
                try:
                    records.append( json.loads(line.strip()) )
                except Exception, e:
                    pass
        return records


def parseAll(paths):
    records = []
    for path in paths:
        records.extend(parseJsonArrayFile(path))
    
    return records

In [6]:
records = parseAll(files)

In [7]:
len(records)

21259

In [8]:
### MY_SCHEMA
'''
id
timestamp (time, unix naive epoch)
geo - lon
geo - lat
text
author uid
retweets
favorites
author_data
application
source
'''

'\nid\ntimestamp (time, unix naive epoch)\ngeo - lon\ngeo - lat\ntext\nauthor uid\nretweets\nfavorites\nauthor_data\napplication\nsource\n'

In [9]:
def parseTweet(tweet, mode='tpl', gnipID='gnip'):
    if 'geo' in tweet:
        if mode=='tpl':
            return (int(tweet['id'].split(':')[-1]),           ## tweet unique id
                    int(dateutil.parser.parse(tweet['postedTime']).strftime("%s")), ## unix naive timestamp
                    tweet['geo']['coordinates'][0],            ## geo - lat
                    tweet['geo']['coordinates'][1],            ## geo - lon
                    tweet['body'],                             ## tweet text body
                    int(tweet['actor']['id'].split(':')[-1]),  ## user id
                    tweet['retweetCount'],                     ## counting retweets
                    tweet['favoritesCount'],                   ## counting favorites
                    tweet['generator']['displayName'],         ## source app
                    gnipID)                                    ## gnip_folder_name (gnip job)
    else:
        pass
#         print tweet.keys()



In [10]:
parcedRecords = [parseTweet(record) for record in records if parseTweet(record, mode='tpl')!=None]

In [11]:
len(parcedRecords)

13029

In [12]:
parcedRecords[110]

(666044874249097216,
 1447650447,
 40.6800804,
 -73.9751587,
 u'Just posted a photo @ Cubana Cafe https://t.co/2nuZYVmCBp',
 316514940,
 0,
 0,
 u'Instagram',
 'gnip')

In [17]:
tweetE = '''CREATE TABLE IF NOT EXISTS tweets (id INTEGER PRIMARY KEY, 
                                               timestamp INTEGER, 
                                               lon REAL, lat REAL, 
                                               tweet TEXT, 
                                               user_id INTEGER, 
                                               rtwts INTEGER,
                                               fvrts INTEGER,
                                               application TEXT,
                                               source TEXT)'''

In [18]:
# usersE = '''CREATE TABLE users (id INTEGER, 
#                                 timestamp INTEGER, raw TEXT, CONSTRAINT pk PRIMARY KEY (id, timestamp))'''

In [19]:
def aggregate(tweets):
    dbPath = path + 'gnip_twitter.db'
    print dbPath
    conn = sqlite3.connect(dbPath)
    c = conn.cursor()
    c.execute(tweetE)
    c.executemany('INSERT OR IGNORE INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tweets )
    conn.commit()
    conn.close()



In [20]:
aggregate(parcedRecords)

/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/gnip/enviroment4gnip/job1/PTDataDownload/downloads/gnip_twitter.db
